In [1]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1NbH-3LL4bqKmunw388J037IJvXlr_kDW' -O 'train_data_lag_5.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1hz_zmgWPN0mSczWV2m4aq9p4NRNdp4rD' -O 'val_data_lag_5.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1v63Zbg5PizgCqhG9FopdO-i1iWuXFo8D' -O 'test_data_lag_5.csv'

--2021-12-21 18:57:55--  https://drive.google.com/uc?export=download&id=1NbH-3LL4bqKmunw388J037IJvXlr_kDW
Resolving drive.google.com (drive.google.com)... 64.233.189.102, 64.233.189.101, 64.233.189.139, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vj2c7n442t8n2hfs3uvjgt6p6pss5672/1640113050000/14724390508833084599/*/1NbH-3LL4bqKmunw388J037IJvXlr_kDW?e=download [following]
--2021-12-21 18:57:58--  https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vj2c7n442t8n2hfs3uvjgt6p6pss5672/1640113050000/14724390508833084599/*/1NbH-3LL4bqKmunw388J037IJvXlr_kDW?e=download
Resolving doc-0g-18-docs.googleusercontent.com (doc-0g-18-docs.googleusercontent.com)... 142.250.157.132, 2404:6800:4008:c13::84
Connecting to doc-0g-18-docs.googleusercontent.com (doc-0

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline
sns.set(color_codes=True)

In [3]:
df_train = pd.read_csv('train_data_lag_5.csv', lineterminator="\n")
df_train.columns=['stock_date', 'movement percent', 'stock_symbol', 'market_binary', 'tweet']
df_train = df_train.drop(columns=['stock_date', 'movement percent', 'stock_symbol'], axis=1)
df_val = pd.read_csv('val_data_lag_5.csv', lineterminator="\n")
df_val.columns=['stock_date', 'movement percent', 'stock_symbol', 'market_binary', 'tweet']
df_val = df_val.drop(columns=['stock_date', 'movement percent', 'stock_symbol'], axis=1)
df_test = pd.read_csv('test_data_lag_5.csv', lineterminator="\n")
df_test.columns=['stock_date', 'movement percent', 'stock_symbol', 'market_binary', 'tweet']
df_test = df_test.drop(columns=['stock_date', 'movement percent', 'stock_symbol'], axis=1)

In [4]:
df_train['market_binary'] = df_train['market_binary'].apply(int)
df_test['market_binary'] = df_test['market_binary'].apply(int)
df_val['market_binary'] = df_val['market_binary'].apply(int)

In [5]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
    text = re.sub(r'http\S+', '', text)
    text = hashtags.sub(' hashtag', text)
    text = mentions.sub(' entity', text)
    return text.strip().lower()

In [6]:
df_train['tweet'] = df_train.tweet.apply(process_text)
df_test['tweet'] = df_test.tweet.apply(process_text)
df_val['tweet'] = df_val.tweet.apply(process_text)

In [7]:
!pip install nltk
!pip install inflect
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [8]:
import nltk
import json 
import inflect
import operator
def mapNounFrequency(sentenceList):
    '''
    This function tags entities for a given list of sentences and returns a 
    frequency map preserving the likeliness of singular and plural occurences.
    '''
    fMap = {}

    if sentenceList == []:
        return fMap
    
    p = inflect.engine()

    for sentence in sentenceList:
        is_noun = lambda pos: pos[:2] == 'NN'
        tokenized = nltk.word_tokenize(sentence)
        entities = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
        for entity in entities.copy():
            entities.remove(entity)
            if not p.singular_noun(entity):
                entities.append(p.plural(entity).lower())
            else:
                entities.append(entity.lower())
        entities = set(entities)

        for entity in entities:
            if entity in fMap:
                fMap[entity] += 1
            else:
                fMap[entity] = 1
    
    frequency = list(fMap.values())
    normalizer = max(frequency)
    for entity,raw in zip(fMap, frequency):
        fMap[entity] = int((raw/normalizer)*100)
    
    return dict( sorted(fMap.items(), key=operator.itemgetter(1),reverse=True))

In [9]:
def process_vectorized_tweets(tweets, d):
  '''
  This function is used here to vectorize the tweets using a customized count vectorizer.
  '''
  vectorized_tweets = []
  for tweet in tweets:
      vectorized_tweet = []
      for word in tweet.split(" "):
        dict_keys = list(d.keys())
        if(word in dict_keys):
          vectorized_tweet.append(dict_keys.index(word) + 1)
        else:
          vectorized_tweet.append(0)
      vectorized_tweets.append(vectorized_tweet)
  return vectorized_tweets

In [10]:
import pickle
vectorized_test_dict = mapNounFrequency(df_test['tweet'].values)
vectorized_test_data = process_vectorized_tweets(df_test['tweet'].values, vectorized_test_dict)
print(len(vectorized_test_data))
with open('test_data.pickle', 'wb') as f:
    pickle.dump(vectorized_test_data, f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


2737


In [11]:
vectorized_val_dict = mapNounFrequency(df_val['tweet'].values)
vectorized_val_data = process_vectorized_tweets(df_val['tweet'].values, vectorized_val_dict)
print(len(vectorized_val_data))
with open('val_data.pickle', 'wb') as f:
    pickle.dump(vectorized_val_data, f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


1981


In [12]:
vectorized_train_dict = mapNounFrequency(df_val['tweet'].values)
vectorized_train_data = process_vectorized_tweets(df_train['tweet'].values, vectorized_train_dict)
print(len(vectorized_train_data))
with open('train_data.pickle', 'wb') as f:
    pickle.dump(vectorized_train_data, f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


14981


In [ ]:
# Use this if the vectorized data is already stored
# import pickle
# with open('test_data.pickle', 'rb') as f:
#     vectorized_test_data = pickle.load(f)
# with open('val_data.pickle', 'rb') as f:
#     vectorized_val_data = pickle.load(f)
# with open('train_data.pickle', 'rb') as f:
#     vectorized_train_data = pickle.load(f)

In [13]:
x_val = np.array(vectorized_val_data)
x_train = np.array(vectorized_train_data)
x_test = np.array(vectorized_test_data)
y_train = np.array(df_train['market_binary'])
y_val = np.array(df_val['market_binary'])
y_test = np.array(df_test['market_binary'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify '

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [16]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
vocab_size = 10000  # Only consider the top 10k words
maxlen = 200  # Considering character limit for tweet
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, dtype='float64')
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen, dtype='float64')
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen, dtype='float64')

In [18]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 32)          326400    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 32)          10656     
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 32)                0     

In [20]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    x_train, y_train, batch_size=32, epochs=100, validation_data=(x_val, y_val)
)

Epoch 1/100
469/469 [==============================] - 59s 121ms/step - loss: 0.6954 - accuracy: 0.5066 - val_loss: 0.6963 - val_accuracy: 0.4563
Epoch 2/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6932 - accuracy: 0.5122 - val_loss: 0.6936 - val_accuracy: 0.4563
Epoch 3/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6939 - accuracy: 0.5071 - val_loss: 0.6906 - val_accuracy: 0.5437
Epoch 4/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6934 - accuracy: 0.5124 - val_loss: 0.6965 - val_accuracy: 0.4563
Epoch 5/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6931 - accuracy: 0.5116 - val_loss: 0.6945 - val_accuracy: 0.4563
Epoch 6/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6930 - accuracy: 0.5136 - val_loss: 0.6962 - val_accuracy: 0.4563
Epoch 7/100
469/469 [==============================] - 56s 120ms/step - loss: 0.6931 - accuracy: 0.5114 - val_loss: 0.6940 -

In [21]:
model.evaluate(x_test, y_test) 

86/86 [==============================] - 4s 47ms/step - loss: 0.6924 - accuracy: 0.5265


[0.6923729181289673, 0.5264888405799866]

In [24]:
predictions = model.predict(x_test)
processed_predictions = []
for item in predictions:
    if item[0] > item[1]:
        processed_predictions.append(0)
    else:
        processed_predictions.append(1)

In [25]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, processed_predictions)

0.0